In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8; py-indent-offset:4 -*-

"""
Strategia di Backtest specifica per verifica performance

STRATEGY TYPE: RSI_MA_Combined
DIRECTION: BUY
INDICATORS: [RSI(28), RSI(21), SMA(100), SMA(50), SMA(200)]

ENTRY CONDITIONS:
  close < SMA(100) and RSI(28) < 30 and close < SMA(100)

EXIT CONDITIONS:
  RSI(21) < 25 and SMA(50) < SMA(200)

DETAILED CONDITIONS:
  Entry Logic: close < SMA(100) and RSI(28) < 30 and close < SMA(100)
  Exit Logic: RSI(21) < 25 and SMA(50) < SMA(200)
"""

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime
import backtrader as bt
import backtrader.feeds as btfeeds


class RSIMAStrategy(bt.Strategy):
    """
    Strategia basata su RSI e Moving Averages
    Entry: close < SMA(100) and RSI(28) < 30
    Exit: RSI(21) < 25 and SMA(50) < SMA(200)
    """
    params = (
        ('rsi_entry_period', 28),  # Periodo RSI per entry
        ('rsi_exit_period', 21),   # Periodo RSI per exit
        ('sma_entry_period', 100), # Periodo SMA per entry
        ('sma_mid_period', 50),    # Periodo SMA medio per exit
        ('sma_long_period', 200),  # Periodo SMA lungo per exit
        ('rsi_entry_threshold', 30), # Soglia RSI per entry
        ('rsi_exit_threshold', 25),  # Soglia RSI per exit
        ('printlog', False),       # Disabilito logging per velocizzare
    )

    def __init__(self):
        # Indicatori tecnici per entry
        self.rsi_entry = bt.talib.RSI(self.data.close, timeperiod=self.p.rsi_entry_period)
        self.sma_entry = bt.talib.SMA(self.data.close, timeperiod=self.p.sma_entry_period)

        # Indicatori tecnici per exit
        self.rsi_exit = bt.talib.RSI(self.data.close, timeperiod=self.p.rsi_exit_period)
        self.sma_mid = bt.talib.SMA(self.data.close, timeperiod=self.p.sma_mid_period)
        self.sma_long = bt.talib.SMA(self.data.close, timeperiod=self.p.sma_long_period)

        # Tracking variabili
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def log(self, txt, dt=None, doprint=False):
        """Logging function for this strategy"""
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    f'BUY EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Cost: {order.executed.value:.2f}, '
                    f'Comm: {order.executed.comm:.2f}')

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log(
                    f'SELL EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Cost: {order.executed.value:.2f}, '
                    f'Comm: {order.executed.comm:.2f}')

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(f'OPERATION PROFIT, GROSS {trade.pnl:.2f}, NET {trade.pnlcomm:.2f}')

    def next(self):
        # Log ridotto per debug
        if len(self) % 100 == 0:  # Log ogni 100 giorni
            self.log(f'Day {len(self)}: Close: {self.data.close[0]:.2f}, '
                    f'RSI28: {self.rsi_entry[0]:.2f}, SMA100: {self.sma_entry[0]:.2f}')

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            # ENTRY CONDITIONS: close < SMA(100) and RSI(28) < 30 and close < SMA(100)
            entry_sma_condition = self.data.close[0] < self.sma_entry[0]
            entry_rsi_condition = self.rsi_entry[0] < self.p.rsi_entry_threshold

            # Combined entry condition (close < SMA(100) appears twice in original, using once)
            if entry_sma_condition and entry_rsi_condition:
                self.log(f'BUY CREATE - Entry conditions met: '
                        f'Close: {self.data.close[0]:.2f} < SMA100: {self.sma_entry[0]:.2f}, '
                        f'RSI28: {self.rsi_entry[0]:.2f} < {self.p.rsi_entry_threshold}')
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:
            # EXIT CONDITIONS: RSI(21) < 25 and SMA(50) < SMA(200)
            exit_rsi_condition = self.rsi_exit[0] < self.p.rsi_exit_threshold
            exit_sma_condition = self.sma_mid[0] < self.sma_long[0]

            if exit_rsi_condition and exit_sma_condition:
                self.log(f'SELL CREATE - Exit conditions met: '
                        f'RSI21: {self.rsi_exit[0]:.2f} < {self.p.rsi_exit_threshold}, '
                        f'SMA50: {self.sma_mid[0]:.2f} < SMA200: {self.sma_long[0]:.2f}')
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.order = self.sell()

    def stop(self):
        self.log(f'Final Portfolio Value: {self.broker.getvalue():.2f}', doprint=True)


def run_backtest():
    """
    Funzione principale per eseguire il backtest
    """
    # Inizializza Cerebro
    cerebro = bt.Cerebro()

    # Carica i dati
    data = btfeeds.GenericCSVData(
        # Using the SPY hourly data that's available in the system
        dataname=r'C:\Users\calli\OneDrive\Programmazione\github\ga\ga\data_csv\h1\spy_h1_2017_2025_formatted.csv',
        dtformat=('%Y-%m-%d %H:%M:%S'),
        datetime=0,
        open=1,
        high=2,
        low=3,
        close=4,
        volume=5,
        openinterest=-1,
        # Date range - utilizziamo tutti i dati disponibili dal 2017 al 2025
        fromdate=datetime.datetime(2017, 1, 20),
        todate=datetime.datetime(2025, 8, 22),
    )

    # Aggiungi i dati a Cerebro
    cerebro.adddata(data)

    # Aggiungi la strategia
    cerebro.addstrategy(RSIMAStrategy)

    # Imposta il capitale iniziale
    cerebro.broker.setcash(100000.0)

    # Imposta la commissione
    cerebro.broker.setcommission(commission=0.001)

    # Aggiungi analyzer per le performance
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trades")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
    cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")

    # Add additional analyzers as requested
    cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name="annual_returns")
    cerebro.addanalyzer(bt.analyzers.TimeDrawDown, _name="time_drawdown")
    cerebro.addanalyzer(bt.analyzers.Returns, _name="detailed_returns")

    # Stampa il valore iniziale del portafoglio
    print(f'Starting Portfolio Value: {cerebro.broker.getvalue():.2f}')

    # Esegui il backtest
    results = cerebro.run()

    # Stampa il valore finale del portafoglio
    print(f'Final Portfolio Value: {cerebro.broker.getvalue():.2f}')

    # Estrai e stampa le statistiche
    strat = results[0]

    print('\n=== TRADE ANALYSIS ===')
    trade_analysis = strat.analyzers.trades.get_analysis()
    if trade_analysis:
        # Gestione sicura dei dati di trade analysis usando try/except
        try:
            total_trades = trade_analysis.total.total
        except (KeyError, AttributeError):
            total_trades = 0
        print(f"Total Trades: {total_trades}")

        # Controllo sicuro per trade vincenti
        won_trades = 0
        gross_profit = 0.0
        try:
            won_trades = trade_analysis.won.total
            try:
                gross_profit = trade_analysis.won.pnl.total
            except (KeyError, AttributeError):
                gross_profit = 0.0
        except (KeyError, AttributeError):
            pass
        print(f"Winning Trades: {won_trades}")

        # Controllo sicuro per trade perdenti
        lost_trades = 0
        gross_loss = 0.0
        try:
            lost_trades = trade_analysis.lost.total
            try:
                gross_loss = trade_analysis.lost.pnl.total
            except (KeyError, AttributeError):
                gross_loss = 0.0
        except (KeyError, AttributeError):
            pass
        print(f"Losing Trades: {lost_trades}")

        print(f"Gross Profit: {gross_profit:.2f}")
        print(f"Gross Loss: {gross_loss:.2f}")

        # Calcolo win rate se ci sono trade
        if total_trades > 0:
            win_rate = (won_trades / total_trades) * 100
            print(f"Win Rate: {win_rate:.2f}%")

        # Profit Factor
        if abs(gross_loss) > 0:
            profit_factor = abs(gross_profit / gross_loss)
            print(f"Profit Factor: {profit_factor:.2f}")

        # # Avg Profit per Trade
        # if total_trades > 0:
        #     avg_profit_per_trade = (gross_profit + gross_loss) / total_trades
        #     print(f"Avg Profit per Trade: {avg_profit_per_trade:.2f}")

        # Net Profit
        net_profit = gross_profit + gross_loss  # gross_loss è già negativo
        print(f"Net Profit: {net_profit:.2f}")
    else:
        print("No trade data available")

    print('\n=== PERFORMANCE METRICS ===')
    sharpe_ratio = strat.analyzers.sharpe.get_analysis()
    if sharpe_ratio and 'sharperatio' in sharpe_ratio:
        print(f"Sharpe Ratio: {sharpe_ratio['sharperatio']:.4f}")

    drawdown = strat.analyzers.drawdown.get_analysis()
    if drawdown:
        print(f"Max Drawdown: {drawdown['max']['drawdown']:.2f}%")
        print(f"Max Drawdown Money: ${drawdown['max']['moneydown']:.2f}")

    returns = strat.analyzers.returns.get_analysis()
    if returns:
        print(f"Total Return: {returns['rtot']*100:.2f}%")
        print(f"Average Return: {returns['ravg']*100:.4f}%")

    # Display results from additional analyzers
    print('\n=== ADDITIONAL ANALYTICS ===')

    # Annual Returns Analysis
    annual_returns = strat.analyzers.annual_returns.get_analysis()
    if annual_returns:
        print("Annual Returns:")
        for year, return_val in annual_returns.items():
            print(f"  {year}: {return_val*100:.2f}%")

    # Time DrawDown Analysis
    time_drawdown = strat.analyzers.time_drawdown.get_analysis()
    if time_drawdown:
        print(f"\nTime DrawDown Analysis:")
        if 'maxdrawdownperiod' in time_drawdown:
            print(f"  Max DrawDown Period: {time_drawdown['maxdrawdownperiod']} days")
        if 'maxdrawdown' in time_drawdown:
            print(f"  Max DrawDown Value: {time_drawdown['maxdrawdown']:.2f}%")

    # Detailed Returns Analysis
    detailed_returns = strat.analyzers.detailed_returns.get_analysis()
    if detailed_returns:
        print(f"\nDetailed Returns Analysis:")
        print(f"  Total Compound Return (rtot): {detailed_returns.get('rtot', 0)*100:.4f}%")
        print(f"  Average Return (ravg): {detailed_returns.get('ravg', 0)*100:.6f}%")
        if 'rnorm' in detailed_returns:
            print(f"  Annualized/Normalized Return (rnorm): {detailed_returns['rnorm']*100:.4f}%")
        if 'rnorm100' in detailed_returns:
            print(f"  Annualized Return in 100% (rnorm100): {detailed_returns['rnorm100']:.4f}%")

    return cerebro, results


if __name__ == '__main__':
    cerebro, results = run_backtest()
